<a href="https://colab.research.google.com/github/ticky1515/Toxic-messages-analysis/blob/main/Machine_Learning_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.utils import np_utils

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# dev version
# !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip

# release version 
! pip install pythainlp

!pip install epitran
!pip install sklearn_crfsuite
!pip install tensorflow deepcut
!pip install attacut

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 52.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10745 sha256=f2e8f161fc17500c951b2ab67ca63bed7b5fdce89620e3ac18f676e73eeea5e6
  Stored in directory: /root/.cache/pip/wheels/9c/ea/66/8e45247b09052a933eb1a680b7c64802298faba58aac9b346b
Successfully built unicodecsv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
import pythainlp

pythainlp.__version__

'4.0.1'

In [ ]:
from pythainlp import sent_tokenize, word_tokenize

In [ ]:
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!gdown --id 1xm6cQHxgodAkSOKvVwWf5d0jPIWqYi2y -O train.csv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1xm6cQHxgodAkSOKvVwWf5d0jPIWqYi2y
To: /content/train.csv
100% 1.41M/1.41M [00:00<00:00, 113MB/s]


In [ ]:
!gdown --id 1H0zxxG1aq-A-jDPyrzm-03JYgqEb5x5r -O test.csv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1H0zxxG1aq-A-jDPyrzm-03JYgqEb5x5r
To: /content/test.csv
100% 585k/585k [00:00<00:00, 118MB/s]


In [ ]:
!gdown --id 1ebPWQNTKQgzE7GiJr3UgFDJ1UiSBETIQ -O train_attacut.csv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ebPWQNTKQgzE7GiJr3UgFDJ1UiSBETIQ
To: /content/train_attacut.csv
100% 1.31M/1.31M [00:00<00:00, 120MB/s]


In [ ]:
!gdown --id 1KaRm6SINFXyKl7mUx0lpZUfpVxxgWC19 -O test_attacut.csv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KaRm6SINFXyKl7mUx0lpZUfpVxxgWC19
To: /content/test_attacut.csv
100% 546k/546k [00:00<00:00, 114MB/s]


In [ ]:
df_train = pd.read_csv('/content/train_attacut.csv')

In [ ]:
df_train = df_train.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df_train

,text_token,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,อห ฝัน ดอกเตอร์ กุ ซื้อ ชิบหาย แม่เอ้ย โหน กระแส,0,0,0,0,0,0
1,จบ เฉลยเนา มู ด็อก บูยอน แม่โซยอน มั้ง รู้ ฆ่า...,0,0,0,0,0,0
2,ทุเร่ดดดดดดดพด ตบ,0,0,0,1,0,0
3,weibo电 视剧星汉灿烂·月 updateofficialรอ รอ รอ รอ รอ ด...,0,0,0,0,0,0
4,หลอด หัด ดูด นั่ง ดูดนอน ดูด อะไหล่ ขวด นม ขวด...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
7118,สมหวัง เย็ด,0,0,1,0,0,0
7119,ดูควย คน,0,0,1,0,0,0
7120,ควย,0,0,1,0,0,0
7121,รัก แบ หยุม หัวแหะ ตอน พี่จิ๋ว ตาย ฝัน ฝัน ตัว...,0,0,0,0,0,0


In [ ]:
df_test = pd.read_csv('/content/test_attacut.csv')

In [ ]:
df_test = df_test.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df_test

,text_token,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,harry ประกาศ เอเชียทัวร์ กุคิด ชุด รอ ละค่า ชุ...,0,0,0,0,0,0
1,ปาก เงียบ โหน กระแส,0,0,0,0,0,0
2,ออกอ่ะยาง อ่ะยาง มั้ง ใจ ท่อง พุท โธ แบ ดยพ่ดก...,0,0,0,0,0,0
3,พ่ก ยิง เรย,0,0,0,0,0,0
4,ราชบุรี ว่าง คิว เย็ด ผม กี่ รอบ สนใจ ทักไลน์p...,0,0,1,0,0,0
...,...,...,...,...,...,...,...
3048,ตัวอย่าง ดี คน ชอบ คหสต ทำร้าย สัก พิสูจน์ ดู ...,0,0,0,0,0,0
3049,ดาราจักรรักลานาใจ ดู เศร้า ตอน ฮองเฮา น้อง สาว...,0,0,0,0,0,0
3050,อ๋อ เฉลย ปะ ตัดต่อ อีควาย ชิป ไล่ แปะ รูป ทวิต...,1,1,0,0,0,0
3051,อารมณ์ หวีด ขนส่ง หัวควย,1,0,0,0,0,0


In [ ]:
X_train = df_train['text_token']
X_test = df_test['text_token']

In [ ]:
y_train = df_train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
y_test = df_test[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

# Y array

In [ ]:
y_train = y_train.values.tolist()

In [ ]:
array_length = len(y_train)
for i in range(array_length):
  print(y_train[i])

Streaming output truncated to the last 5000 lines.
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]

In [ ]:
y_train = np.array(y_train)

In [ ]:
y_train

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
y_test = y_test.values.tolist()

In [ ]:
array_length = len(y_test)
for i in range(array_length):
  print(y_test[i])

[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 1]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0,

In [ ]:
y_test = np.array(y_test)

In [ ]:
y_test

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [1, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Model

Random forest

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_names = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier()
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)


In [ ]:
#RandomForestClassifier
from sklearn.metrics import classification_report
target_names = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
print(classification_report(y_test, y_pred, target_names=target_names))

               precision    recall  f1-score   support

        toxic       0.83      0.59      0.69       593
 severe_toxic       0.60      0.07      0.12        46
      obscene       0.92      0.80      0.86       694
       threat       0.38      0.10      0.16        48
       insult       0.72      0.26      0.38       238
identity_hate       0.56      0.22      0.32        68

    micro avg       0.86      0.59      0.70      1687
    macro avg       0.67      0.34      0.42      1687
 weighted avg       0.82      0.59      0.67      1687
  samples avg       0.32      0.30      0.30      1687



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf_DeTree = DecisionTreeClassifier(random_state=0)
clf_DeTree.fit(X_train_tfidf, y_train)
y_pred_DeTree = clf_DeTree.predict(X_test_tfidf)

In [ ]:
#DecisionTreeClassifier PRED
print(classification_report(y_test, y_pred_DeTree, target_names=target_names))

               precision    recall  f1-score   support

        toxic       0.75      0.64      0.69       593
 severe_toxic       0.36      0.26      0.30        46
      obscene       0.81      0.81      0.81       694
       threat       0.21      0.15      0.17        48
       insult       0.51      0.41      0.45       238
identity_hate       0.33      0.24      0.28        68

    micro avg       0.72      0.64      0.67      1687
    macro avg       0.50      0.42      0.45      1687
 weighted avg       0.70      0.64      0.67      1687
  samples avg       0.32      0.31      0.31      1687



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Extratree

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
ensemble_ExtraTree = ExtraTreesClassifier()
ensemble_ExtraTree.fit(X_train_tfidf, y_train)
y_pred_ensemble_ExtraTree = ensemble_ExtraTree.predict(X_test_tfidf)

In [ ]:
#EnsembleExtraTreeClassifier PRED
print(classification_report(y_test, y_pred_ensemble_ExtraTree, target_names=target_names))

               precision    recall  f1-score   support

        toxic       0.82      0.58      0.68       593
 severe_toxic       0.71      0.11      0.19        46
      obscene       0.94      0.80      0.86       694
       threat       0.43      0.12      0.19        48
       insult       0.69      0.28      0.40       238
identity_hate       0.54      0.22      0.31        68

    micro avg       0.86      0.59      0.70      1687
    macro avg       0.69      0.35      0.44      1687
 weighted avg       0.83      0.59      0.67      1687
  samples avg       0.31      0.30      0.30      1687



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# criterion = ["gini", "entropy", "log_loss"]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
class_weight = ["balanced", "balanced_subsample", None]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
              #  'criterion' : criterion,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'class_weight': class_weight}
               
print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'class_weight': ['balanced', 'balanced_subsample', None]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
extr = ExtraTreesClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
extr_random = RandomizedSearchCV(estimator = extr, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
extr_random.fit(X_train_tfidf, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomizedSearchCV(cv=3, estimator=ExtraTreesClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': ['balanced',
                                                         'balanced_subsample',
                                                         None],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100, 1200,
                                                         1300, 1400, 1500, 1600,
                                                         1700, 1800, 1900,
                                                         2000]},
                   random_state=42, verbose=2)

In [ ]:
extr_random.best_params_

{'n_estimators': 1200,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'class_weight': 'balanced_subsample',
 'bootstrap': False}

In [ ]:
# {'n_estimators': 1366,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 'sqrt',
#  'max_depth': None,
#  'class_weight': None,
#  'bootstrap': True}

In [ ]:
best_random_ensem_extratree = extr_random.best_estimator_
y_pred_ensem_extratree_tuned = best_random_ensem_extratree.predict(X_test_tfidf)
print(classification_report(y_test, y_pred_ensem_extratree_tuned, target_names=target_names))

               precision    recall  f1-score   support

        toxic       0.80      0.62      0.70       593
 severe_toxic       0.75      0.13      0.22        46
      obscene       0.93      0.77      0.84       694
       threat       0.13      0.31      0.19        48
       insult       0.62      0.31      0.41       238
identity_hate       0.42      0.25      0.31        68

    micro avg       0.77      0.60      0.67      1687
    macro avg       0.61      0.40      0.45      1687
 weighted avg       0.79      0.60      0.67      1687
  samples avg       0.32      0.30      0.31      1687



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
neigh.fit(X_train_tfidf, y_train)
y_pred_neigh = neigh.predict(X_test_tfidf)

In [ ]:
#KNeighborsClassifier PRED
print(classification_report(y_test, y_pred_neigh, target_names=target_names))

               precision    recall  f1-score   support

        toxic       0.69      0.23      0.34       593
 severe_toxic       1.00      0.07      0.12        46
      obscene       0.91      0.32      0.48       694
       threat       0.25      0.15      0.18        48
       insult       0.63      0.18      0.28       238
identity_hate       1.00      0.04      0.08        68

    micro avg       0.76      0.25      0.37      1687
    macro avg       0.75      0.16      0.25      1687
 weighted avg       0.78      0.25      0.37      1687
  samples avg       0.13      0.13      0.13      1687



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
               
print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_tfidf, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100, 1200,
                                                         1300, 1400, 1500, 1600,
                                                         1700, 1800, 1900,
                                                         2000]},
                   random_state=42, verbose=2)

In [ ]:
rf_random.best_params_

{'n_estimators': 500,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [ ]:
#RandomForestClassifier after tuned
from sklearn.metrics import classification_report
best_random = rf_random.best_estimator_
y_pred_randomF_tuned = best_random.predict(X_test_tfidf)
print(classification_report(y_test, y_pred_randomF_tuned, target_names=target_names))

               precision    recall  f1-score   support

        toxic       0.82      0.62      0.70       593
 severe_toxic       0.67      0.09      0.15        46
      obscene       0.92      0.80      0.85       694
       threat       0.40      0.12      0.19        48
       insult       0.69      0.27      0.39       238
identity_hate       0.54      0.22      0.31        68

    micro avg       0.85      0.60      0.70      1687
    macro avg       0.67      0.35      0.43      1687
 weighted avg       0.82      0.60      0.68      1687
  samples avg       0.32      0.30      0.31      1687



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
